In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation as animation
from sklearn import datasets  # to retrieve the iris Dataset
import pandas as pd  # to load the dataframe
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA
import seaborn as sns  # to plot the heat maps#float32
from numba import jit, cuda
from numba import jit, prange

In [2]:
# Load spike data
#WF_data = np.load('data/data_WF_short.npy')
WF_data = np.memmap('data/data_WF_short.npy',dtype='float32',mode='r', shape=(313600, 5000))
#wf_space_data_temp = np.memmap('data/svdSpatialComponents.npy', )

#WF_data_t = WF_data[:,10000:10999]
print(np.shape(WF_data))

W_short = np.array(WF_data[:,:1000])
W = np.array(WF_data)
# short time span
#t = np.shape(spike_data_t)[1]
#n = np.shape(spike_data)[0]

(313600, 5000)


In [3]:
@jit(target_backend='cuda')
def scaling(a):
    #scalar = StandardScaler()
    #scaled_data = pd.DataFrame(scalar.fit_transform(a)) #scaling the data
    #print(scaled_data.shape)
    
    a = (a - np.mean(a,axis=0))/ np.std(a, axis = 0)

    #Applying PCA

    #Taking no. of Principal Components r
    #r = 500
    #pca = PCA(n_components = r)

    #data_pca = pca.fit_transform(a) # = US

    #c = pca.components_      # V
    #s = pca.singular_values_ # S

    return a

In [4]:
@jit(target_backend='cuda')
def scaling_numba(a):

    res = []
    for i in prange(a.shape[0]):
        res.append(a[i, :].mean())

    return np.array(res)


In [5]:
@jit(target_backend='cuda')
def scaling_numba1(a):

    mean = np.zeros((1,a.shape[1]))
    
    for j in prange(a.shape[1]):
        for i in prange(a.shape[0]):
            mean[0,j] = mean[0,j] + a[i,j]
        
        mean[0,j] = mean[0,j]/a.shape[0]
        
    std = np.zeros((1,a.shape[1]))
    for j in prange(a.shape[1]):
        for i in prange(a.shape[0]):
            std[0,j] = np.sqrt((a[i,j] - mean[0,j])**2/a.shape[0])
    
    for j in prange(a.shape[1]):
        a[:,j] = (a[:,j]-mean[0,j])/std[0,j]

    return a

In [6]:
#W_scaled = scaling(W_short)
#W_scaled = scaling(W)
W_scaled = scaling_numba1(W)

In [7]:
print(W_scaled.shape)

(313600, 5000)


In [8]:
@jit(target_backend='cuda')
def PCA_scale(a):
    #scalar = StandardScaler()
    #scaled_data = pd.DataFrame(scalar.fit_transform(a)) #scaling the data
    #print(scaled_data.shape)
    
    u, s, v = np.linalg.svd(a, full_matrices=False)

    #Applying PCA

    #Taking no. of Principal Components r
    #r = 500
    #pca = PCA(n_components = r)

    #data_pca = pca.fit_transform(a) # = US

    #c = pca.components_      # V
    #s = pca.singular_values_ # S

    return u,s,v

In [ ]:
[u,s,v] = PCA_scale(W_scaled)

In [ ]:
print(u.shape)
print(s.shape)
print(v.shape)
fig, ax = plt.subplots()
ax.plot(s)

ax.set(xlabel='rank', ylabel='S',
       title='Singular values')
plt.show()

In [ ]:
print(s[:100])

In [ ]:
# projection
r=20
u_r = u[:,:r]
s_r = s[:r]
S = np.diag(s_r)

#_proj = u_r.T@W_short
W_proj = u_r.T@W_scaled

#W_proj = np.linalg.inv(S)@u_r.T@W_short

In [ ]:
%matplotlib qt
dims=20
t=1000
time = np.linspace(1, t, t)
d = np.linspace(1, dims, dims)
T, N = np.meshgrid(time, d)

fig = plt.figure(figsize = (12,10))
ax = plt.axes(projection='3d')

surf = ax.plot_surface(T, N, W_proj[0:dims,:], cmap = plt.cm.cividis)
# Set axes label
#ax.set_xlabel('x', labelpad=20)
#ax.set_ylabel('y', labelpad=20)
#ax.set_zlabel('z', labelpad=20)

fig.colorbar(surf, shrink=0.5, aspect=8)

plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
ax.plot(W_proj[:2,:].T)
plt.show()

In [ ]:
print(W_proj.shape)

In [ ]:
# reconstruction
@jit(target_backend='cuda')
def PCA_recon(a,u_r):
    
    U = u_r@u_r.T
    b = U@a



    return b